In [136]:
import simpy
import random

In [137]:
class Servers:
    def __init__(self, env, mu):
        self.mu = mu
        self.server = simpy.Resource(env, capacity=mu)

def task(name, env, servers):

    # A task arrives at the server
    print(f'Task {name} arriving at {env.now}')

    # Request task to server
    with servers.server.request() as req:

        # Once there is availability at the server, initiate task
        yield req
        print(f'Server starts processing {name} at {env.now}')

        # Calculate time at server by Poisson process, not sure if necessary
        processing_time = int(random.expovariate(1 / servers.mu))
        yield env.timeout(processing_time)
        print(f'Server done processing {name} at {env.now}')


def task_generator(env, servers, N, lambda_):

    # Generate N tasks
    for i in range(N):

        # Create task
        env.process(task(i, env, servers))

        # Calculate time until next task by Poisson process, not sure if necessary
        next_arrival = int(random.expovariate(lambda_))
        yield env.timeout(next_arrival)

mu = 2
lambda_ = 0.8
N = 10
tmax = 20

env = simpy.Environment()
servers = Servers(env, mu)
car_gen = env.process(task_generator(env, servers, N, lambda_))
env.run(tmax)

Task 0 arriving at 0
Task 1 arriving at 0
Server starts processing 0 at 0
Task 2 arriving at 0
Server starts processing 1 at 0
Server done processing 0 at 0
Task 3 arriving at 0
Task 4 arriving at 0
Server starts processing 2 at 0
Task 5 arriving at 0
Server done processing 2 at 0
Task 6 arriving at 0
Server starts processing 3 at 0
Server done processing 3 at 0
Server starts processing 4 at 0
Server done processing 1 at 2
Task 7 arriving at 2
Server done processing 4 at 2
Server starts processing 5 at 2
Server starts processing 6 at 2
Server done processing 6 at 3
Server starts processing 7 at 3
Task 8 arriving at 4
Server done processing 5 at 4
Server done processing 7 at 4
Server starts processing 8 at 4
Task 9 arriving at 5
Server starts processing 9 at 5
Server done processing 8 at 7
Server done processing 9 at 8
